In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/token_id/oauth2/token"}


dbutils.fs.mount(
source = "abfss://tokyoolympicdata@tokyoolympicadls.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/tokyoolympic",
extra_configs = configs)

True

In [0]:
%fs ls "/mnt/tokyoolympic"

path,name,size,modificationTime
dbfs:/mnt/tokyoolympic/raw_data/,raw_data/,0,1733402223000
dbfs:/mnt/tokyoolympic/transformed_data/,transformed_data/,0,1733402232000


In [0]:
%fs ls "/mnt/tokyoolympic/raw_data"

path,name,size,modificationTime
dbfs:/mnt/tokyoolympic/raw_data/Athelet.csv,Athelet.csv,407406,1733404082000
dbfs:/mnt/tokyoolympic/raw_data/Coaches.csv,Coaches.csv,16494,1733404097000
dbfs:/mnt/tokyoolympic/raw_data/Entriesgender.csv,Entriesgender.csv,1076,1733404115000
dbfs:/mnt/tokyoolympic/raw_data/Medals.csv,Medals.csv,2320,1733404150000
dbfs:/mnt/tokyoolympic/raw_data/Teams.csv,Teams.csv,34526,1733404168000


In [0]:
from pyspark.sql.functions import col, count, round
from pyspark.sql.window import Window

In [0]:
Athelet_df = spark.read.format('csv').option('header', True).option('inferSchema', True).load('/mnt/tokyoolympic/raw_data/Athelet.csv')
Athelet_df = Athelet_df.fillna({"Country": "Unknown", "Discipline": "Unknown"})
Athelet_df.printSchema()
Athelet_df.count()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = false)
 |-- Discipline: string (nullable = false)



11085

In [0]:
Athelet_df = Athelet_df.drop_duplicates(subset=['PersonName', 'Country', 'Discipline'])
Athelet_df.count()

11084

In [0]:
# Add a new column 'Total_persons_per_country' means the number of athletes in each country

windowspec = Window.partitionBy('Country')
Athelet_df = Athelet_df.withColumn('Total_persons_per_country', count('PersonName').over(windowspec))

# Add a new column 'Total_persons_per_country_per_discipline' means the number of athletes in each country and same discipline in same country
windowspec_new = Window.partitionBy('Country', 'Discipline')
Athelet_df = Athelet_df.withColumn('Total_persons_per_country_per_discipline', count('PersonName').over(windowspec_new))


In [0]:
coaches_df = spark.read.format('csv').option('header', True).option('inferSchema', True).load('/mnt/tokyoolympic/raw_data/Coaches.csv')
coaches_df.printSchema()
coaches_df.count()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



394

In [0]:
coaches_df = coaches_df.drop_duplicates(subset=['Name', 'Country', 'Discipline', 'Event'])
coaches_df.count()

393

In [0]:
coaches_df = coaches_df.fillna({"Country": "Unknown", "Discipline": "Unknown", "Event": "Unknown"})

In [0]:
windowspec_1 = Window.partitionBy("Country", "Discipline")
coaches_df = coaches_df.withColumn("Total_coaches_per_country_per_discipline", count("Name").over(windowspec_1))

In [0]:
entriesgender_df = spark.read.format('csv').option('header', True).option('inferSchema', True).load('/mnt/tokyoolympic/raw_data/Entriesgender.csv')
entriesgender_df.printSchema()
entriesgender_df.count()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



46

In [0]:
entriesgender_df = entriesgender_df.withColumn(
    'Avg_Female', round(entriesgender_df['Female'] / entriesgender_df['Total'], 2)
).withColumn(
    'Avg_Male', round(entriesgender_df['Male'] / entriesgender_df['Total'], 2)
)
entriesgender_df.show()

+--------------------+------+----+-----+----------+--------+
|          Discipline|Female|Male|Total|Avg_Female|Avg_Male|
+--------------------+------+----+-----+----------+--------+
|      3x3 Basketball|    32|  32|   64|       0.5|     0.5|
|             Archery|    64|  64|  128|       0.5|     0.5|
| Artistic Gymnastics|    98|  98|  196|       0.5|     0.5|
|   Artistic Swimming|   105|   0|  105|       1.0|     0.0|
|           Athletics|   969|1072| 2041|      0.47|    0.53|
|           Badminton|    86|  87|  173|       0.5|     0.5|
|   Baseball/Softball|    90| 144|  234|      0.38|    0.62|
|          Basketball|   144| 144|  288|       0.5|     0.5|
|    Beach Volleyball|    48|  48|   96|       0.5|     0.5|
|              Boxing|   102| 187|  289|      0.35|    0.65|
|        Canoe Slalom|    41|  41|   82|       0.5|     0.5|
|        Canoe Sprint|   123| 126|  249|      0.49|    0.51|
|Cycling BMX Frees...|    10|   9|   19|      0.53|    0.47|
|  Cycling BMX Racing|  

In [0]:
medals_df = spark.read.format('csv').option('header', True).option('inferSchema', True).load('/mnt/tokyoolympic/raw_data/Medals.csv')
medals_df = medals_df.orderBy('Rank by Total')
medals_df.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   3|               Japan|  27|    14|    17|   58|            5|
|   6|           Australia|  17|     7|    22|   46|            6|
|  10|               Italy|  10|    10|    20|   40|            7|
|   9|             Germany|  10|    11|    16|   37|            8|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [0]:
teams_df = spark.read.format('csv').option('header', True).option('inferSchema', True).load('/mnt/tokyoolympic/raw_data/Teams.csv')
teams_df.show()

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [0]:
Athelet_df.write.mode('overwrite').option('header', True).csv('/mnt/tokyoolympic/transformed_data/Athelets')


In [0]:
coaches_df.write.mode('overwrite').option('header', True).csv('/mnt/tokyoolympic/transformed_data/Coaches')
entriesgender_df.write.mode('overwrite').option('header', True).csv('/mnt/tokyoolympic/transformed_data/EntriesGender')
medals_df.write.mode('overwrite').option('header', True).csv('/mnt/tokyoolympic/transformed_data/Medals')
teams_df.write.mode('overwrite').option('header', True).csv('/mnt/tokyoolympic/transformed_data/Teams')